# Training a FOM model
This notebook will train a first order motion model based on a self-constructed dataset.

* First we need to preprocess the data in the form that is required by the project.

* Then start the training process. The training requires the corresponding `*.yaml` config file, and remember to properly set the config file. In this part, I'll try to start train from a pre-trained model.

## Data Preparation

In [9]:
# Data Preparation
import os

raw_path = "../facial_dataset/data/clips"
target_path = "data/anim-256"

if not os.path.exists(target_path):
    os.mkdir(target_path)

# create train/test folder in target folder
if not os.path.exists(target_path + '/train'):
    os.mkdir(target_path + '/train')
if not os.path.exists(target_path + '/test'):
    os.mkdir(target_path + '/test')

In [13]:
# Generater filenames that will be copied into the folder
import shutil
import random

def rand_generator(list_length, num_rand):
        t = list(range(list_length))
        random.shuffle(t)
        return t[0:num_rand]

train_path = target_path + '/train'
test_path = target_path + '/test'

num_files = len(os.listdir(train_path)) # decide the new file name starts with which number
num_tests = len(os.listdir(test_path))
src_files = []

# get the list of source file path
film_list = os.listdir(raw_path)
print(film_list)
for folder in film_list:
    if len(folder) < 4:
        continue
    if folder[0: 4] != "film":
        continue
    temp_path = raw_path + '/' + folder
    clips = os.listdir(temp_path)
    for clip in clips:
        if clip[-3:] != 'mp4':
            continue
        if clip[0] == '.':
            continue
        src_path = temp_path + '/' + clip
        src_files.append(src_path)

# separate 0.1 of all files as test files
print("Num sources: {}".format(len(src_files)))
test_index = rand_generator(len(src_files), 26)

['.DS_Store', '.ipynb_checkpoints', '._.DS_Store', 'film1', 'film10', 'film11', 'film12', 'film13', 'film14', 'film15', 'film16', 'film2', 'film3', 'film4', 'film5', 'film6', 'film7', 'film8', 'film9']
Num sources: 296


In [14]:
# Copy files
for i, src in enumerate(src_files):
    if i not in test_index:
        target_name = train_path + '/train_{}'.format(num_files)
        # Here can have more options
        # Now just copy .mp4 files
        # As instructed, we can generate folders containing .png of each frame in the video
        shutil.copy(src, target_name + '.mp4')
        num_files += 1
    else:
        target_name = test_path + '/test_{}'.format(num_tests)
        shutil.copy(src, target_name + '.mp4')
        num_tests += 1

## Train section
Now data is prepared. We need to train a FOM model.

In [8]:
!pwd
#!python run.py --config config/anim-256.yaml

'pwd' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


## Label Preparation

In [6]:
# Data Preparation
import os

raw_path = "../facial_dataset/data/test/vox2_test_mp4/mp4"
target_path = "data/real-256"

if not os.path.exists(target_path):
    os.mkdir(target_path)

# create train/test folder in target folder
if not os.path.exists(target_path + '/train'):
    os.mkdir(target_path + '/train')
if not os.path.exists(target_path + '/test'):
    os.mkdir(target_path + '/test')

In [7]:
# Generater filenames that will be copied into the folder
import shutil
import random

def rand_generator(list_length, num_rand):
        t = list(range(list_length))
        random.shuffle(t)
        return t[0:num_rand]

train_path = target_path + '/train'
test_path = target_path + '/test'

num_files = len(os.listdir(train_path)) # decide the new file name starts with which number
num_tests = len(os.listdir(test_path))
src_files = []

# get the list of source file path
film_list = os.listdir(raw_path)
print(len(film_list))
for folder in film_list:
    if len(folder) < 4:
        continue
    if folder[0: 2] != "id":
        continue
    temp_path = raw_path + '/' + folder
    subfolders = os.listdir(temp_path)
    # loop over the secondary directories
    for sub in subfolders:
        if sub[0] == '.':
            continue
        final_path = temp_path + '/' + sub
        clips = os.listdir(final_path)
        for clip in clips:
            if clip[-3:] != 'mp4':
                continue
            if clip[0] == '.':
                continue
            src_path = final_path + '/' + clip
            src_files.append(src_path)

# separate 0.1 of all files as test files
print("Num sources: {}".format(len(src_files)))
pick_index = rand_generator(len(src_files), 1000)
print("Num of picked sources: {}".format(len(pick_index)))
test_index = rand_generator(len(pick_index), 100)

118
Num sources: 36237
Num of picked sources: 1000


In [8]:
import cv2

# Copy files
for i, src in enumerate(pick_index):
    if i not in test_index:
        target_name = train_path + '/train_{}'.format(num_files)
        # Here can have more options
        # Now just copy .mp4 files
        # As instructed, we can generate folders containing .png of each frame in the video
        videoWriter = cv2.VideoWriter(target_name + '.mp4', cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 24, (256, 256))
        cap = cv2.VideoCapture(src_files[src])
        frame_index = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (256, 256))
            videoWriter.write(frame)
        videoWriter.release()
        num_files += 1
    else:
        target_name = test_path + '/test_{}'.format(num_tests)
        videoWriter = cv2.VideoWriter(target_name + '.mp4', cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 24, (256, 256))        
        cap = cv2.VideoCapture(src_files[src])
        frame_index = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (256, 256))
            videoWriter.write(frame)
        videoWriter.release()
        num_tests += 1